**Jasmin Putri Jelita - 202110370311075**

**Tazkia Damayanti - 202110370311090**

**UAS TEMU KEMBALI INFORMASI**

In [32]:
import os
import re
import json
import pandas as pd

In [7]:
# Load keywords from a file
def load_keywords_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keywords = json.load(file)
    return keywords

# Extract "No Putusan"
def extract_no_putusan(content):
    pattern = r"(Nomor\s*[:\-]?\s*\d+/[A-Za-z.]+/\d+/[A-Za-z\s\-]{1,7})"
    match = re.search(pattern, content)
    return match.group(1) if match else None

# Extract "Lembaga Peradilan"
def extract_lembaga_peradilan(content):
    pattern = r"(Pengadilan\s*(Negeri|Agama|Tinggi)(\s*di)?\s*\w+)(?!\w)"
    match = re.search(pattern, content)
    return match.group(1) if match else None

# Extract data from content using keywords
def extract_data_from_content(content, keywords):
    extracted_texts = []
    extracted_ranges = []

    for keyword in keywords:
        pattern = re.escape(keyword)
        match = re.search(pattern, content, re.IGNORECASE)

        if match:
            start_index = match.start()
            end_index = content.find('\n\n', start_index)
            end_index = end_index if end_index != -1 else len(content)

            overlaps = any(start <= start_index <= end or start <= end_index <= end for start, end in extracted_ranges)
            if overlaps:
                continue

            extracted_paragraph = content[start_index:end_index].strip()

            while extracted_paragraph.count('.') < 5:
                additional_end_index = content.find('\n\n', end_index + 2)
                additional_end_index = additional_end_index if additional_end_index != -1 else len(content)
                extracted_paragraph += content[end_index:additional_end_index].strip()
                end_index = additional_end_index

            extracted_texts.append(extracted_paragraph)
            extracted_ranges.append((start_index, end_index))

    consolidated_text = "\n\n".join(extracted_texts)
    return consolidated_text if consolidated_text else None

In [8]:
# Extract data from a file
def extract_data_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    extracted_data = {category: None for category in KEYWORDS.keys()}

    # Ekstraksi data wajib
    extracted_data["No Putusan"] = extract_no_putusan(content) or "Data tidak ditemukan"
    extracted_data["Lembaga Peradilan"] = extract_lembaga_peradilan(content) or "Data tidak ditemukan"
    extracted_data["Barang Bukti"] = extract_data_from_content(content, KEYWORDS.get("Barang Bukti", [])) or "Data tidak ditemukan"
    extracted_data["Amar Putusan"] = extract_data_from_content(content, KEYWORDS.get("Amar Putusan", [])) or "Data tidak ditemukan"

    # Ekstraksi data lainnya
    for category, keywords in KEYWORDS.items():
        if category not in ["No Putusan", "Lembaga Peradilan", "Barang Bukti", "Amar Putusan"]:
            extracted_data[category] = extract_data_from_content(content, keywords) or "Data tidak ditemukan"

    return extracted_data

In [34]:
def clean_illegal_characters(data):
    cleaned_data = {k: re.sub(r'[\x00-\x1F\x7F]', ' ', v) if v else v for k, v in data.items()}
    return cleaned_data

def save_to_excel(data, output_file):
    cleaned_data = {filename: clean_illegal_characters(file_data) for filename, file_data in data.items()}
    df = pd.DataFrame(cleaned_data).transpose()
    df = df.fillna("False")
    df.to_excel(output_file, index_label="File Name")

def read_and_extract_data(output_file):
    base_folder = os.path.join(os.getcwd(), "txt_putusan_UAS")  # Subfolder dari root direktori
    all_extracted_data = {}
    for file_name in os.listdir(base_folder):
        if file_name.endswith('.txt'):
            file_path = os.path.join(base_folder, file_name)
            all_extracted_data[file_name] = extract_data_from_file(file_path)

    save_to_excel(all_extracted_data, output_file)

In [36]:
if __name__ == "__main__":
    keywords_file_path = "keywords.txt"
    output_file = "Data.xlsx"

    KEYWORDS = load_keywords_from_file(keywords_file_path)

    if not KEYWORDS:
        print("Kata kunci tidak dapat dimuat. Periksa file keywords.txt")
    else:
        read_and_extract_data(output_file)
        print(f"Data telah diekstraksi dan disimpan ke {output_file}.")

Data telah diekstraksi dan disimpan ke Data.xlsx.
